# goormEDU Crawling Program


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By


page = 1
url = 'https://edu.goorm.io/category/programming?page=1&sort=accuracy'

driver = webdriver.Chrome()

driver.get(url)
driver.get(url)

soup = BeautifulSoup(driver.page_source,'html.parser')


In [8]:
#'강의명', '강의자', '온라인강의 소요시간', '교육과정', '가격', '평균평점'
res_goorm = pd.DataFrame(columns=['강의명', '강의자', '온라인강의 소요시간', '교육과정', '가격', '평균평점'])

In [6]:
#강의명
soup.select('div._1v8zxY mb-1')[0]

IndexError: list index out of range

In [13]:
#강의자
soup.select('div._3M41pF')[0].text

IndexError: list index out of range

In [ ]:
#온라인 강의 소요시간

In [ ]:
#교육과정

In [7]:
#가격
soup.select('span._1zPZlD')[0].text
prices =[p.text for p in soup.select('span._1zPZlD')]


['무료',
 '무료',
 '무료',
 '무료',
 '무료',
 '무료',
 '무료',
 '무료',
 '₩30,000',
 '₩23,100',
 '₩23,100',
 '₩33,000',
 '무료',
 '무료',
 '₩199,000',
 '₩199,000',
 '₩77,000',
 '₩55,000',
 '₩55,000',
 '₩44,000']

In [ ]:
#평균평점

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def get_course_info(course_url):
    response = requests.get(course_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 강좌 소개, 교육 과정, 교육 후기 등의 정보를 추출
    intro = soup.find('div', class_='course-intro').text.strip()
    curriculum = soup.find('div', class_='course-curriculum').text.strip()
    reviews_tag = soup.find('div', class_='course-reviews')
    if reviews_tag:
        reviews = reviews_tag.text.strip()
    else:
        reviews = "수강 후기가 없습니다."
    
    return intro, curriculum, reviews

def scrape_goorm_edu():
    base_url = "https://edu.goorm.io/category/programming?page={}&sort=accuracy"
    all_course_info = []
    
    for page in range(1, 24):  # 페이지: 1부터 23까지
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 각 페이지의 강의 리스트에서 강의명, 강의자, 가격을 추출
        courses = soup.find_all('div', class_='course-detail')
        for course in courses:
            title = course.find('div', class_='course-title').text.strip()
            instructor = course.find('div', class_='instructor').text.strip()
            price = course.find('div', class_='price').text.strip()
            course_url = "https://edu.goorm.io" + course.find('a')['href']
            
            # 각각의 강의 페이지에서 강좌 소개, 교육 과정, 교육 후기를 가져오기
            intro, curriculum, reviews = get_course_info(course_url)
            
            all_course_info.append({
                "Title": title,
                "Instructor": instructor,
                "Price": price,
                "Intro": intro,
                "Curriculum": curriculum,
                "Reviews": reviews
            })
            
            # 잠시 대기하여 서버부하 줄이기
            time.sleep(1)
    
    return all_course_info

def save_to_csv(course_info, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Title', 'Instructor', 'Price', 'Intro', 'Curriculum', 'Reviews']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for course in course_info:
            writer.writerow(course)

if __name__ == "__main__":
    # Goorm Edu 사이트에서 강의 정보를 크롤링하기
    all_course_info = scrape_goorm_edu()
    
    # CSV 파일로 저장합니다.
    save_to_csv(all_course_info, 'goorm_edu_courses.csv')
    
    print("크롤링이 완료되었습니다.")
